In [1]:
# import libraries

import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [55]:
cap.release()
cv2.destroyAllWindows()

In [2]:
# index of landmarks in right and left eye
# horizontal line for left eye: [0, 8]
# first vertical line: [3, 13]
# second vertical line: [5, 11]
left_eye_lm = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7]
# horizontal line for right eye: [0, 8]
# first vertical line: [3, 13]
# second vertical line: [5, 11]
right_eye_lm = [263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249]

In [3]:
def eye_coords(frame, landmarks):
    height, width, _ = frame.shape
    eye_x = []
    eye_y = []
    for lm in landmarks:
        # mp return normalized coords, so we multiply
        eye_x.append(int((results.face_landmarks.landmark[lm].x) * width))
        eye_y.append(int((results.face_landmarks.landmark[lm].y) * height))
    eye = list(zip(eye_x, eye_y))
    eye = np.array(eye, dtype="int")
    return eye

In [4]:
def eye_aspect_ratio(eye):
    # euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[3], eye[13])
    B = dist.euclidean(eye[5], eye[11])
    # euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[8])
    # eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

In [8]:
# initialize counters and threshold
frame_counter = 0
ear_frame_counter = 0
ear_threshold = []

cap = cv2.VideoCapture("../media/video/distracted_6min.mp4")
#cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        # update frame counter
        #frame_counter += 1
        height, width, _ = frame.shape
        # Convert the BGR image to RGB.
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        frame.flags.writeable = False
        
        try:
            results = holistic.process(frame)
            
            if results.face_landmarks:
                frame_counter += 1
            
            # get eyes coords
            left_eye = eye_coords(frame, left_eye_lm)
            right_eye = eye_coords(frame, right_eye_lm)

            # get EARs
            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)

            # average of both EARs
            ear = (left_ear + right_ear) / 2

            # find convex hull for the eyes   
            leye_hull = cv2.convexHull(left_eye)
            reye_hull = cv2.convexHull(right_eye)

            # Draw convex hull
            frame.flags.writeable = True
            cv2.drawContours(frame, [leye_hull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [reye_hull], -1, (0, 255, 0), 1)

            # take some time to initialize values
            if frame_counter < 30:
                cv2.putText(frame, "CALIBRATING", (20, 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))
                ear_threshold.append(ear)
            else:
                # convert ear_threshold to numpy array
                ear_threshold = np.array(ear_threshold)
               # remove NaN values from the array
                ear_threshold = ear_threshold[~np.isnan(ear_threshold)] # ~ = is not
                # get the mean
                ear_threshold = np.mean(np.array(ear_threshold))

                # print EAR on the frame
                cv2.putText(frame, f"EAR: {round(ear, 2)}", (20, 20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))

                # check if EAR goes below the threshold for a number of frames
                if ear < ear_threshold:
                    ear_frame_counter += 1

                    if ear_frame_counter >= 10:
                        print("EAR " + str(ear))
                        print("THRESH " + str(ear_threshold))
                        print("drowsy")
                        cv2.putText(frame, "WARNING!", (20, 100), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))

                else:
                    ear_frame_counter = 0
        except:
            cv2.putText(frame, "NO DETECTION", (20, 100), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255))
            pass

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        
        cv2.imshow('Webcam Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

EAR 0.33590322082967117
THRESH 0.4049158575571246
drowsy
EAR 0.3276545328840951
THRESH 0.4049158575571246
drowsy
EAR 0.3327873253089
THRESH 0.4049158575571246
drowsy
EAR 0.32481991555613055
THRESH 0.4049158575571246
drowsy
EAR 0.34176009976231897
THRESH 0.4049158575571246
drowsy
EAR 0.3285634724931103
THRESH 0.4049158575571246
drowsy
EAR 0.3536097678951536
THRESH 0.4049158575571246
drowsy
EAR 0.3334218178127748
THRESH 0.4049158575571246
drowsy
EAR 0.3573144214569142
THRESH 0.4049158575571246
drowsy
EAR 0.3488053641127804
THRESH 0.4049158575571246
drowsy
EAR 0.36935119493155033
THRESH 0.4049158575571246
drowsy
EAR 0.33865530613420514
THRESH 0.4049158575571246
drowsy
EAR 0.34710301610601557
THRESH 0.4049158575571246
drowsy
EAR 0.35615508100719306
THRESH 0.4049158575571246
drowsy
EAR 0.3667903989504436
THRESH 0.4049158575571246
drowsy
EAR 0.3594140559334716
THRESH 0.4049158575571246
drowsy
EAR 0.3536097678951536
THRESH 0.4049158575571246
drowsy
EAR 0.3587252914407043
THRESH 0.404915857557

EAR 0.2913910599127053
THRESH 0.4049158575571246
drowsy
EAR 0.292999137585254
THRESH 0.4049158575571246
drowsy
EAR 0.2715444463147443
THRESH 0.4049158575571246
drowsy
EAR 0.3267891721664421
THRESH 0.4049158575571246
drowsy
EAR 0.2918794069598851
THRESH 0.4049158575571246
drowsy
EAR 0.28870432694441917
THRESH 0.4049158575571246
drowsy
EAR 0.3539958366650189
THRESH 0.4049158575571246
drowsy
EAR 0.3285427990787002
THRESH 0.4049158575571246
drowsy
EAR 0.31460651190456673
THRESH 0.4049158575571246
drowsy
EAR 0.2954565712692887
THRESH 0.4049158575571246
drowsy
EAR 0.3298427504241117
THRESH 0.4049158575571246
drowsy
EAR 0.3175851797462335
THRESH 0.4049158575571246
drowsy
EAR 0.3381021063109374
THRESH 0.4049158575571246
drowsy
EAR 0.3627702338444845
THRESH 0.4049158575571246
drowsy
EAR 0.2811409234031431
THRESH 0.4049158575571246
drowsy
EAR 0.3059071679230279
THRESH 0.4049158575571246
drowsy
EAR 0.3882913731575753
THRESH 0.4049158575571246
drowsy
EAR 0.3304820863480625
THRESH 0.404915857557124

EAR 0.30876389286006983
THRESH 0.4049158575571246
drowsy
EAR 0.32646863347563626
THRESH 0.4049158575571246
drowsy
EAR 0.25752672546252686
THRESH 0.4049158575571246
drowsy
EAR 0.2399061891193623
THRESH 0.4049158575571246
drowsy
EAR 0.2924723708034781
THRESH 0.4049158575571246
drowsy
EAR 0.281359018298764
THRESH 0.4049158575571246
drowsy
EAR 0.2525422363041044
THRESH 0.4049158575571246
drowsy
EAR 0.28677507768526056
THRESH 0.4049158575571246
drowsy
EAR 0.2664962121699178
THRESH 0.4049158575571246
drowsy
EAR 0.275089734879953
THRESH 0.4049158575571246
drowsy
EAR 0.2806464887512806
THRESH 0.4049158575571246
drowsy
EAR 0.3006112709885664
THRESH 0.4049158575571246
drowsy
EAR 0.2826447404588184
THRESH 0.4049158575571246
drowsy
EAR 0.29493736057370834
THRESH 0.4049158575571246
drowsy
EAR 0.29801493879145236
THRESH 0.4049158575571246
drowsy
EAR 0.3154179622644472
THRESH 0.4049158575571246
drowsy
EAR 0.29117976877001134
THRESH 0.4049158575571246
drowsy
EAR 0.3339416230866226
THRESH 0.40491585755

EAR 0.21160335726263912
THRESH 0.4049158575571246
drowsy
EAR 0.3592266381268412
THRESH 0.4049158575571246
drowsy
EAR 0.16805955271749473
THRESH 0.4049158575571246
drowsy
EAR 0.20332476267715885
THRESH 0.4049158575571246
drowsy
EAR 0.108887186020518
THRESH 0.4049158575571246
drowsy
EAR 0.26084408833907047
THRESH 0.4049158575571246
drowsy
EAR 0.34075732072050074
THRESH 0.4049158575571246
drowsy
EAR 0.40324113519317567
THRESH 0.4049158575571246
drowsy
EAR 0.28833755187279775
THRESH 0.4049158575571246
drowsy
EAR 0.18984177641647657
THRESH 0.4049158575571246
drowsy
EAR 0.31025400900051586
THRESH 0.4049158575571246
drowsy
EAR 0.20791904232231048
THRESH 0.4049158575571246
drowsy
EAR 0.2033598395046508
THRESH 0.4049158575571246
drowsy
EAR 0.18401775389094285
THRESH 0.4049158575571246
drowsy
EAR 0.21117315813523183
THRESH 0.4049158575571246
drowsy
EAR 0.2274522993967481
THRESH 0.4049158575571246
drowsy
EAR 0.2033598395046508
THRESH 0.4049158575571246
drowsy
EAR 0.1771064998239584
THRESH 0.40491